In [1]:
import os
os.chdir('../../..')

In [2]:
import convokit

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from convokit import Corpus, download

In [4]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [5]:
corpus.print_summary_stats()

Number of Users: 7568
Number of Utterances: 74467
Number of Conversations: 10744


## Some new Conversation functionality

In [6]:
convo = corpus.get_conversation('o31u0')

In [7]:
convo.print_conversation_structure()

cchambo
    jklol
    djnap
    Brimwoodboy
        jklol


In [8]:
convo.print_conversation_structure(lambda utt: utt.id)

o31u0
    c3dzmtu
    c3e0ou0
    c3f7l5b
        c3feqc4


In [9]:
convo.get_chronological_utterance_list()

[Utterance({'id': 'o31u0', 'user': User([('name', 'cchambo')]), 'root': 'o31u0', 'reply_to': None, 'timestamp': 1325714498, 'text': '', 'meta': {'score': 27, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/o31u0/cornell_scientists_create_hole_in_time_where/', 'author_flair_text': 'SNES 2015'}}),
 Utterance({'id': 'c3dzmtu', 'user': User([('name', 'jklol')]), 'root': 'o31u0', 'reply_to': 'o31u0', 'timestamp': 1325721216, 'text': 'wooo! aep!', 'meta': {'score': 3, 'top_level_comment': 'c3dzmtu', 'retrieved_on': 1428124647, 'gilded': 0, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '', 'author_flair_text': 'AEP 2011'}}),
 Utterance({'id': 'c3e0ou0', 'user': User([('name', 'djnap')]), 'root': 'o31u0', 'reply_to': 'o31u0', 'timestamp': 1325727478, 'text': "it's always nice to see Cornell in the news", 'meta': {'score': 2, 'top_level_comment': 'c3e0ou0', 're

In [10]:
[utt.user.id for utt in convo.get_chronological_utterance_list()]

['cchambo', 'jklol', 'djnap', 'Brimwoodboy', 'jklol']

In [11]:
convo.get_root_to_leaf_paths()

[[Utterance({'id': 'o31u0', 'user': User([('name', 'cchambo')]), 'root': 'o31u0', 'reply_to': None, 'timestamp': 1325714498, 'text': '', 'meta': {'score': 27, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/o31u0/cornell_scientists_create_hole_in_time_where/', 'author_flair_text': 'SNES 2015'}}),
  Utterance({'id': 'c3dzmtu', 'user': User([('name', 'jklol')]), 'root': 'o31u0', 'reply_to': 'o31u0', 'timestamp': 1325721216, 'text': 'wooo! aep!', 'meta': {'score': 3, 'top_level_comment': 'c3dzmtu', 'retrieved_on': 1428124647, 'gilded': 0, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '', 'author_flair_text': 'AEP 2011'}})],
 [Utterance({'id': 'o31u0', 'user': User([('name', 'cchambo')]), 'root': 'o31u0', 'reply_to': None, 'timestamp': 1325714498, 'text': '', 'meta': {'score': 27, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': Non

In [12]:
for path in convo.get_root_to_leaf_paths():
    print([utt.user.id for utt in path])

['cchambo', 'jklol']
['cchambo', 'Brimwoodboy', 'jklol']
['cchambo', 'djnap']


## Cumulative BoW

In [13]:
from convokit import Forecaster

Let's set up a forecasting task to predict for whether a Reddit comment will have a positive score, i.e. upvotes > downvotes.

In [14]:
# Adding a 'y' feature to fit to
for utt in corpus.iter_utterances():
    utt.add_meta('pos_score', int(utt.meta['score'] > 0))

In [15]:
forecaster = Forecaster(pred_feat='pos_score', skip_broken_convos=True)

No model passed to Forecaster. Initializing default forecaster model: Cumulative Bag-of-words...
Initializing default unigram CountVectorizer...
Initializing default classification model (standard scaled logistic regression)


In [16]:
forecaster.fit(corpus)

Fitting cumulative BoW classification model...
Done.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [21]:
forecast_df = forecaster.analyze(corpus)

In [23]:
forecast_df.head(10)

,forecast,forecast_prob
id,,
c3ditbx,1,0.964617
ctzwhsb,1,0.998855
cu6j5ri,1,0.997771
ctzg5r7,1,0.997771
cu00g2f,1,0.997771
cudy1e2,1,0.992397
cu13u5g,1,0.987555
cu0wkg8,1,0.987555
cu2hgyp,1,0.997665


In [27]:
forecast_df.tail(10)

,forecast,forecast_prob
id,,
drzg4es,0,0.257587
djhndfz,0,0.385630
drzt9m8,0,0.171797
dfjdhf4,0,0.383082
dfjg69w,0,0.490404
dfjft8p,0,0.383082
dpn8e4v,0,0.315501
ds0hc2x,0,0.182581
dpnld62,0,0.172047


In [18]:
forecaster.transform(corpus)

Let's examine a Conversation that has an utterance forecasted to have negative score.

In [28]:
corpus.get_utterance('dpn8e4v')

Utterance({'id': 'dpn8e4v', 'user': User([('name', 'Trumpsamerican')]), 'root': '7a75x0', 'reply_to': 'dpm8anu', 'timestamp': 1510358978, 'text': "If you don't mind me asking, is your gpa over a 3.7? Because I heard that ILR's average GPA is a 3.5, and that's incredibly low given that History majors typically get 3.8+. I applied to ILR, btw.\n\nLastly, pertaining to the last portion of your statement, is it possible to take only HR classes, and things in that realm, and avoid history/law classes?", 'meta': {'score': 0, 'top_level_comment': 'dp95zls', 'retrieved_on': 1512515249, 'gilded': 0, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/7a75x0/what_is_the_best_college_for_law_school/dpn8e4v/', 'author_flair_text': '', 'pos_score': 0, 'forecast': 0.0, 'forecast_prob': 0.3155005792126009}})

In [30]:
corpus.get_utterance('dpn8e4v').root

'7a75x0'

In [32]:
corpus.get_conversation(corpus.get_utterance('dpn8e4v').root).print_conversation_structure()

Trumpsamerican
    IthacaisGorges_
    _vpl
    lyfehack
        Trumpsamerican
            byanilla
                Trumpsamerican
                    byanilla
                        Trumpsamerican
                            byanilla
                                Trumpsamerican
                                    byanilla
                                        Trumpsamerican
                                            byanilla
                    [deleted]
    mattezai


### Forecasted

In [34]:
corpus.get_conversation(corpus.get_utterance('dpn8e4v').root).print_conversation_structure(lambda utt: str(utt.meta['forecast']))

None
    1.0
    1.0
    1.0
        1.0
            1.0
                0.0
                    1.0
                        0.0
                            1.0
                                1.0
                                    1.0
                                        1.0
                                            1.0
                    1.0
    1.0


### Actual

In [35]:
corpus.get_conversation(corpus.get_utterance('dpn8e4v').root).print_conversation_structure(lambda utt: str(utt.meta['pos_score']))

0
    1
    1
    1
        1
            1
                0
                    1
                        0
                            1
                                1
                                    1
                                        1
                                            1
                    1
    1


In [24]:
forecasts = [utt.meta['forecast'] for utt in corpus.iter_utterances()]
actual = [utt.meta['pos_score'] for utt in corpus.iter_utterances()]

In [36]:
y_true_pred = [(forecast, actual) for forecast, actual in zip(forecasts, actual) if forecast is not None]

In [52]:
import numpy as np
from collections import Counter

In [54]:
y_pred = np.array([x[0] for x in y_true_pred])

In [55]:
y_true = np.array([x[1] for x in y_true_pred])

In [56]:
# baseline accuracy: assume all positive
np.mean(y_true)

0.9413316168736361

In [57]:
# achieved accuracy
np.mean(y_true == y_pred)

0.9527607265648304

In [58]:
from sklearn.metrics import confusion_matrix

In [60]:
confusion_matrix(y_true=y_true, y_pred=y_pred)

array([[  893,  2844],
       [  165, 59795]])